In [ ]:
!pip install xlrd

In [ ]:
!pip install xlrd==1.2.0

In [ ]:
!pip install openpyxl

In [ ]:
!pip install pandas 

In [ ]:
!pip install pymongo

In [1]:
import pandas as pd 
from pymongo import MongoClient

In [2]:
url = "mongodb://localhost:27017/"
conn = MongoClient(url)
db = conn['zameen']

print('Data connection is successful')

Data connection is successful


In [238]:
# resp = db['plots_data'].remove({})

/Users/mustafa/Workspace/Pandas/Notebooks/env/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.


In [166]:
# resp = db['rental_plots_data'].remove({})

/Users/mustafa/Workspace/Pandas/Notebooks/env/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.


In [229]:
query = {
#     'city' : 'Islamabad',
#     'area' : '5 Marla'
}

df = pd.DataFrame(list(db.plots_data.find()))

In [230]:
len(df)

1430

In [11]:
df['price'].str[:-4].head()

0       54 
1       60 
2       45 
3     1.3 C
4    1.25 C
Name: price, dtype: object

In [12]:
df['city'].value_counts()

Lahore             502
Islamabad          475
Karachi            430
Karachi_Clifton     25
Name: city, dtype: int64

In [261]:
df.pivot_table(index='_id', columns='city', values='area', aggfunc='sum').head()

In [20]:
df.pivot_table(index='area', columns='city', values='price', aggfunc='first')

In [ ]:
# Convert price column to one unit e.g crore or lakh or 1000 
# cast column to integer

# Convert Area column to one unit e.g crore or lakh or 1000 
# cast column to integer

In [ ]:
# bar graph of cities on x axis
# y axis shows count of 5 marla, 10 marla, 20 marla and so on


# Min, max and average price range for all cities

# show correlation between price and area
# show corelation between price and city



In [226]:
def convert_price_to_numeric(price_str):
    if 'Thousand' in price_str:
        return float(price_str.replace('Thousand', ''))/100
    
    if 'Lakh' in price_str:
        return float(price_str.replace(' Lakh', ''))
    
    elif 'Crore' in price_str:
        return float(price_str.replace(' Crore', '')) * 100
    
    elif 'Arab' in price_str:
        return float(price_str.replace(' Arab', '')) * 10000
    
    else:
        return price_str


In [ ]:
# Apply the conversion function to the 'price' column
df['price_in_numeric'] = df['price'].apply(convert_price_to_numeric)

In [23]:
df['Plot Price (in Lakh)'] = df['price_in_numeric'].map(int)

In [29]:
def convert_to_kanals(area):
    area = area.replace(',', '')  # Remove comma from the area string
    
    if 'Kanal' in area:
        return float(area.split()[0]) 
    
    elif 'Marla' in area:
        return float(area.split()[0]) / 20
    
    elif 'Sq. Yd.' in area:
        return float(area.split()[0]) / 20
    
    else:
        return None

In [30]:
df['area_in_kanals'] = df['area'].apply(convert_to_kanals)

In [ ]:
# to convert square yard to marla i.e sq_yd/20.004
def square_yards_to_marla(square_yards):
    return square_yards / 20.0004

square_yards_to_marla(1000)

In [31]:
def convert_to_marla(area):
    area = area.replace(',', '')  # Remove comma from the area string
    
    if 'Kanal' in area:
        return float(area.split()[0]) * 20
    
    elif 'Marla' in area:
        return float(area.split()[0])
    
    elif 'Sq. Yd.' in area:
        return float(area.split()[0]) / 20
    
    else:
        return None

In [32]:
df['area_in_marla'] = df['area'].apply(convert_to_marla)

In [35]:
mini_df = df[["city","area_in_marla","Plot Price (in Lakh)"]]

In [262]:
df.pivot_table(index="city", columns="area_in_marla", aggfunc="count")

In [37]:
correlation = df['price_in_numeric'].corr(df['area_in_kanals'])

print("Correlation between 'price_in_numeric' and 'area_in_kanals':", correlation)

("Correlation between 'price_in_numeric' and 'area_in_kanals':", 0.337091228204013)


In [38]:
# df = df[['city', 'location', 'area_in_marla', 'Plot Price (in Lakh)', ]]

In [ ]:
# data = 'plots_data_update.xlsx'
# df.to_excel(data)

In [240]:
some_df = pd.read_excel('plots_data_update.xlsx', 'Sheet1')

In [241]:
df = some_df

In [8]:
some_df['bucket_1'] = pd.qcut(some_df['Plot Price (in Lakh)'], q=10)

In [10]:
some_df['bucket_2'] = pd.qcut(some_df['Plot Price (in Lakh)'], q=10, precision=0)

In [43]:
some_df['marla_bucket'] = pd.qcut(some_df['area_in_marla'], q=4, precision=0)

In [23]:
# some_df.to_excel("spreadsheets/working.xlsx", index=False)

In [206]:
rental_plot_df =pd.DataFrame(list(db.rental_plots_data.find()))                 

In [244]:
rental_plot_df['price_in_numeric'] = rental_plot_df['Price'].apply(convert_price_to_numeric)

In [246]:
rental_plot_df['area_in_marla'] = rental_plot_df['Area'].apply(convert_to_marla)

In [51]:
rental_plot_df = rental_plot_df[['city', 'location', 'area_in_marla', 'price_in_numeric']]

In [250]:
rental_plot_df.rename(columns={'price_in_numeric' : 'Plot Price (in Lakh)'}, inplace=True)

### To insert Data in Mongo Collection After Transformation

In [258]:
payload = []

for index, row in df.iterrows():
    data_dict = row.to_dict() 
    data_dict['type'] = 'Residential'
    payload.append(data_dict)

In [260]:
resp = db.plots_data.insert(payload)

/Users/mustafa/Workspace/Pandas/Notebooks/env/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """Entry point for launching an IPython kernel.


In [251]:
rental_plot_df.rename(columns= {'Area' : 'area', 'Location' : 'location', 'City' : 'city'}, inplace=True)

In [253]:
rental_plot_df = rental_plot_df[['city', 'location', 'area_in_marla', 'Plot Price (in Lakh)', 'Total Number of Beds', 'Total Number of Baths']]

In [254]:
# for rental data
rental_payload = []

for index, row in rental_plot_df.iterrows():
    data_dict = row.to_dict() 
    data_dict['type'] = 'Rental'
    rental_payload.append(data_dict)

In [256]:
rental_resp = db.plots_data.insert(rental_payload)

/Users/mustafa/Workspace/Pandas/Notebooks/env/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """Entry point for launching an IPython kernel.


In [263]:
len(pd.DataFrame(db.rental_plots_data.find({})))

In [92]:
resp = db.plots_data.insert_many(sample_data_2)

In [95]:
# query = {
#     "type" : "Rental"
# }

# resp = db.plots_data.remove(query)
# resp

/Users/mustafa/Workspace/Pandas/Notebooks/env/lib/python2.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """


{u'n': 9, u'ok': 1.0}

In [205]:
# resp = db['plots_data'].remove({'type' : 'Rental'})

In [4]:
df = pd.DataFrame(list(db.plots_data.find({})))

In [5]:
df.head()

,Plot Price (in Lakh),Total Number of Baths,Total Number of Beds,_id,area_in_marla,city,location,type
0,0.6,4,3,64c15b38d884c0fc3b5881c2,5.0,Lahore,"Bahria Town - Block BB, Bahria Town - Sector D",Rental
1,2.3,6,5,64c15b38d884c0fc3b5881c3,20.0,Lahore,"DHA Phase 5, DHA Defence",Rental
2,2.3,6,5,64c15b38d884c0fc3b5881c4,20.0,Lahore,"DHA Phase 6, DHA Defence",Rental
3,2.3,6,5,64c15b38d884c0fc3b5881c5,20.0,Lahore,"DHA Phase 4, DHA Defence",Rental
4,2.3,6,5,64c15b38d884c0fc3b5881c6,20.0,Lahore,"DHA Phase 5, DHA Defence",Rental


In [6]:
df['type'].value_counts()

Rental         1600
Residential    1430
Name: type, dtype: int64

In [11]:
pivot_table = pd.pivot_table(df, index=["city", "type"], values=[ "Plot Price (in Lakh)"], aggfunc="mean")
pivot_table

Plot Price (in Lakh)
city            type                             
Islamabad       Rental                   6.756411
                Residential            235.353684
Karachi         Rental                   2.765421
                Residential            306.839535
Karachi_Clifton Rental                   6.428916
                Residential           3413.680000
Lahore          Rental                   2.315734
                Residential            279.724000